# Types and Typeclasses

Haskell has a static type system. The type of every expression is known at compile time, which leads to safer code. Haskell has type inference. If we write a number, we don't have to tell Haskell it's a number. It can infer that on its own, so we don't have to explicitly write out the types of our functions and expressions to get things done.

In [1]:
:t 'a'

'a' :: Char

In [2]:
:t (True, 'a')

(True, 'a') :: (Bool, Char)

Functions also have types. When writing our own functions, we can choose to give them an explicit type declaration.

In [3]:
removeNonUppercase :: [Char] -> [Char]  
removeNonUppercase st = [ c | c <- st, c `elem` ['A'..'Z']]

Common Types:
Int, Integer (not bounded), Float, Double, Bool, Char

### Type Variables

In [4]:
:t head

head :: forall a. [a] -> a

a is actually a type variable. That means that a can be of any type. This is much like generics in other languages, only in Haskell it's much more powerful because it allows us to easily write very general functions if they don't use any specific behavior of the types in them. Functions that have type variables are called polymorphic functions. The type declaration of head states that it takes a list of any type and returns one element of that type.

### Typeclasses
A typeclass is a sort of interface that defines some behavior. If a type is a part of a typeclass, that means that it supports and implements the behavior the typeclass describes.

In [5]:
:t (==)

(==) :: forall a. Eq a => a -> a -> Bool

The Eq typeclass provides an interface for testing for equality. Any type where it makes sense to test for equality between two values of that type should be a member of the Eq class.

Some basic typeclasses:
Eq, Ord, Show, Read, Enum, Bounded, Num, Integral, Floating

In [6]:
read "5"

: 

That's why we can use explicit type annotations. Type annotations are a way of explicitly saying what the type of an expression should be. We do that by adding :: at the end of the expression and then specifying a type.

In [7]:
read "5" :: Int

5

# Syntax in Functions

## Pattern Matching
Pattern matching consists of specifying patterns to which some data should conform and then checking to see if it does and deconstructing the data according to those patterns.

In [8]:
factorial :: (Integral a) => a -> a  
factorial 0 = 1  
factorial n = n * factorial (n - 1)

In [11]:
capital :: String -> String  
capital "" = "Empty string, whoops!"  
capital all@(x:xs) = "The first letter of " ++ all ++ " is " ++ [x]

In [12]:
capital "Dracula"

"The first letter of Dracula is D"

## Guards
Whereas patterns are a way of making sure a value conforms to some form and deconstructing it, guards are a way of testing whether some property of a value (or several of them) are true or false.

In [ ]:
bmiTell :: (RealFloat a) => a -> String  
bmiTell bmi  
    | bmi <= 18.5 = "You're underweight, you emo, you!"  
    | bmi <= 25.0 = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | bmi <= 30.0 = "You're fat! Lose some weight, fatty!"  
    | otherwise   = "You're a whale, congratulations!"

### Where
We put the keyword where after the guards (usually it's best to indent it as much as the pipes are indented) and then we define several names or functions. These names are visible across the guards and give us the advantage of not having to repeat ourselves.

In [15]:
bmiTell :: (RealFloat a) => a -> a -> String  
bmiTell weight height  
    | bmi <= skinny = "You're underweight, you emo, you!"  
    | bmi <= normal = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | bmi <= fat    = "You're fat! Lose some weight, fatty!"  
    | otherwise     = "You're a whale, congratulations!"  
    where bmi = weight / height ^ 2  
          skinny = 18.5  
          normal = 25.0  
          fat = 30.0

### Let
Very similar to where bindings are let bindings. Where bindings are a syntactic construct that let you bind to variables at the end of a function and the whole function can see them, including all the guards. Let bindings let you bind to variables anywhere and are expressions themselves, but are very local, so they don't span across guards. The difference is that let bindings are expressions themselves. where bindings are just syntactic constructs.

In [17]:
cylinder :: (RealFloat a) => a -> a -> a  
cylinder r h = 
    let sideArea = 2 * pi * r * h  
        topArea = pi * r ^2  
    in  sideArea + 2 * topArea  

### Case


In [18]:
head' :: [a] -> a  
head' xs = case xs of [] -> error "No head for empty lists!"  
                      (x:_) -> x 

# Recursion

In [19]:
replicate' :: (Num i, Ord i) => i -> a -> [a]  
replicate' n x  
    | n <= 0    = []  
    | otherwise = x:replicate' (n-1) x 

Num is not a subclass of Ord. That means that what constitutes for a number doesn't really have to adhere to an ordering. So that's why we have to specify both the Num and Ord class constraints when doing addition or subtraction and also comparison.

In [20]:
 Quicksort
quicksort :: (Ord a) => [a] -> [a]  
quicksort [] = []  
quicksort (x:xs) =   
    let smallerSorted = quicksort [a | a <- xs, a <= x]  
        biggerSorted = quicksort [a | a <- xs, a > x]  
    in  smallerSorted ++ [x] ++ biggerSorted 

# Higher Order Functions
Haskell functions can take functions as parameters and return functions as return values. A function that does either of those is called a higher order function.

## Curried Functions
Every function in Haskell officially only takes one parameter.
Putting a space between two things is simply function application. The space is sort of like an operator and it has the highest precedence. Let's examine the type of max. It's max :: (Ord a) => a -> a -> a. That can also be written as max :: (Ord a) => a -> (a -> a). That could be read as: max takes an a and returns (that's the ->) a function that takes an a and returns an a. That's why the return type and the parameters of functions are all simply separated with arrows.
If we call a function with too few parameters, we get back a partially applied function, meaning a function that takes as many parameters as we left out. Using partial application (calling functions with too few parameters, if you will) is a neat way to create functions on the fly so we can pass them to another function or to seed them with some data.

In [22]:
compareWithHundred :: (Num a, Ord a) => a -> Ordering  
compareWithHundred = compare 100

In [ ]:
divideByTen :: (Floating a) => a -> a  
divideByTen = (/10)

In [26]:
multThree :: (Num a) => a -> a -> a -> a  
multThree x y z = x * y * z
multTwoWithNine = multThree 9
multTwoWithNine 3 4

108

In [24]:
multThree 3 4

: 

GHCI is telling us that the expression produced a function of type a -> a but it doesn't know how to print it to the screen. Functions aren't instances of the Show typeclass, so we can't get a neat string representation of a function. When we do, say, 1 + 1 at the GHCI prompt, it first calculates that to 2 and then calls show on 2 to get a textual representation of that number. And the textual representation of 2 is just the string "2", which then gets printed to our screen.

In [28]:
applyTwice :: (a -> a) -> a -> a  
applyTwice f x = f (f x)
applyTwice (+3) 10
applyTwice (3:) [1]

16

[3,3,1]

## Maps and filters
Map takes a function and a list and applies that function to every element in the list, producing a new list.
Filter is a function that takes a predicate and a list and then returns the list of elements that satisfy the predicate.

In [29]:
map (+3) [1,5,3,1,6]
filter (`elem` ['A'..'Z']) "i lauGh At You BecAuse u r aLL the Same"

[4,8,6,4,9]

"GAYBALLS"

In [30]:
quicksort :: (Ord a) => [a] -> [a]    
quicksort [] = []    
quicksort (x:xs) =     
    let smallerSorted = quicksort (filter (<=x) xs)  
        biggerSorted = quicksort (filter (>x) xs)   
    in  smallerSorted ++ [x] ++ biggerSorted 

## Lambdas
Lambdas are basically anonymous functions that are used because we need some functions only once. Normally, we make a lambda with the sole purpose of passing it to a higher-order function.

In [31]:
map (\(a,b) -> a + b) [(1,2),(3,5),(6,3),(2,6),(2,5)]

Line 1: Use uncurry
Found:
\ (a, b) -> a + b
Why not:
uncurry (+)

[3,8,9,8,7]

## Folds
A fold takes a binary function, a starting value and a list to fold up.
The foldl function folds the list up from the left side.

In [36]:
sum' :: (Num a) => [a] -> a  
sum' = foldl (+) 0

Line 2: Use sum
Found:
foldl1 (+)
Why not:
sum

# Making custom Types and Typeclasses

In [42]:
data Point = Point Float Float deriving (Show)  
data Shape = Circle Point Float | Rectangle Point Point deriving (Show)

In [43]:
:t Circle
:t Rectangle
-- Value constructors

Circle :: Point -> Float -> Shape

Rectangle :: Point -> Point -> Shape

In [44]:
data Person = Person { firstName :: String  
                     , lastName :: String  
                     , age :: Int  
                     , height :: Float  
                     , phoneNumber :: String  
                     , flavor :: String  
                     } deriving (Show)

In [45]:
:t flavor
:t Person

flavor :: Person -> String

Person :: String -> String -> Int -> Float -> String -> String -> Person

## Type Parameters
Type constructors can take types as parameters to produce new types.

In [50]:
data Maybe a = Nothing | Just a deriving (Show)

The a here is the type parameter. And because there's a type parameter involved, we call Maybe a type constructor. Depending on what we want this data type to hold when it's not Nothing, this type constructor can end up producing a type of Maybe Int, Maybe Car, Maybe String, etc. No value can have a type of just Maybe, because that's not a type per se, it's a type constructor.

In [55]:
:t Just "HAHA"

Just "HAHA" :: Maybe [Char]

## Derived Instances

In [56]:
data Person = Person { firstName :: String  
                     , lastName :: String  
                     , age :: Int  
                     } deriving (Eq, Show, Read)

If a type's constructors have fields, their type has to be a part of derived instances if we want to make our type an instance of them. String, Int part of Eq, Show, Read.

## Type Synonym

In [57]:
type PhoneNumber = String
type Name = String
type PhoneBook = [(Name,PhoneNumber)]

In [58]:
data Either a b = Left a | Right b deriving (Eq, Ord, Read, Show)

we usually use the result type of Either a b, where a is some sort of type that can tell us something about the possible failure and b is the type of a successful computation. Hence, errors use the Left value constructor while results use Right.

## Recursive Data Structures
We can create recursive data types, where one value of some type contains values of that type, which in turn contain more values of the same type and so on.

In [59]:
data List a = Empty | Cons a (List a) deriving (Show, Read, Eq, Ord)

## Defining custom typeclasses

In [ ]:
class Eq a where
    (==) :: a -> a -> Bool
    (/=) :: a -> a -> Bool
    x == y = not (x /= y)
    x /= y = not (x == y)

In [61]:
data TrafficLight = Red | Yellow | Green

In [62]:
instance Eq TrafficLight where
    Red == Red = True
    Green == Green = True
    Yellow == Yellow = True
    _ == _ = False

In [ ]:
instance Show TrafficLight where
    show Red = "Red light"
    show Yellow = "Yellow light"
    show Green = "Green light"

### Functor typeclass
Functor typeclass, which is basically for things that can be mapped over.

In [ ]:
class Functor f where  
    fmap :: (a -> b) -> f a -> f b

In [ ]:
instance Functor [] where  
    fmap = map

# Input and Output

In [63]:
main = putStrLn "hello, world"

In [65]:
:t putStrLn

putStrLn :: String -> IO ()

The empty tuple is a value of () and it also has a type of ().

In [67]:
:t getLine

getLine :: IO String

getLine is an I/O action that contains a result type of String. That makes sense, because it will wait for the user to input something at the terminal and then that something will be represented as a string.

In [68]:
nameTag = "Hello, my name is " ++ getLine

: 

The reason that this doesn't work is that ++ requires both its parameters to be lists over the same type. The left parameter has a type of String (or [Char] if you will), whilst getLine has a type of IO String. You can't concatenate a string and an I/O action. We first have to get the result out of the I/O action to get a value of type String and the only way to do that is to say something like name <- getLine inside some other I/O action. If we want to deal with impure data, we have to do it in an impure environment.

In [72]:
main = do   
    line <- getLine  
    if null line  
        then return ()  
        else do  
            putStrLn $ reverseWords line  
            main  
  
reverseWords :: String -> String  
reverseWords = unwords . map reverse . words

In imperative languages, return usually ends the execution of a method or subroutine and makes it report some sort of value to whoever called it. In Haskell (in I/O actions specifically), it makes an I/O action out of a pure value. If you think about the box analogy from before, it takes a value and wraps it up in a box. The resulting I/O action doesn't actually do anything, it just has that value encapsulated as its result. So in an I/O context, return "haha" will have a type of IO String. What's the point of just transforming a pure value into an I/O action that doesn't do anything? Why taint our program with IO more than it has to be? Well, we needed some I/O action to carry out in the case of an empty input line. That's why we just made a bogus I/O action that doesn't do anything by writing return ().

Using return doesn't cause the I/O do block to end in execution

Some I/O functions: putStr, putStrLn, putChar, getLine, print, when, sequence, forever

## Files and Streams

getChar; getLine; getContents is an I/O action that reads everything from the standard input until it encounters an eof, lazy I/O.

...

# Functors, Applicative Functors and Monoids

## Functors redux
Functors are things that can be mapped over, like lists, Maybes, trees, and such. In Haskell, they're described by the typeclass Functor, which has only one typeclass method, namely fmap, which has a type of fmap :: (a -> b) -> f a -> f b. It says: give me a function that takes an a and returns a b and a box with an a (or several of them) inside it and I'll give you a box with a b (or several of them) inside it. It kind of applies the function to the element inside the box.
Instances of Functors: IO String, (->) r
...

## Applicative Functors
The Applicative typeclass lies in the Control.Applicative module and it defines two methods, pure and <*>. It doesn't provide a default implementation for any of them, so we have to define them both if we want something to be an applicative functor. The class is defined like so:

In [ ]:
class (Functor f) => Applicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b

It says that if we want to make a type constructor part of the Applicative typeclass, it has to be in Functor first.
pure should take a value of any type and return an applicative functor with that value inside it. We take a value and we wrap it in an applicative functor that has that value as the result inside it.
Whereas fmap takes a function and a functor and applies the function inside the functor, <*> takes a functor that has a function in it and another functor and sort of extracts that function from the first functor and then maps it over the second one.

Applicative functors and the applicative style of doing pure f <*> x <*> y <*> ... allow us to take a function that expects parameters that aren't necessarily wrapped in functors and use that function to operate on several values that are in functor contexts. The function can take as many parameters as we want, because it's always partially applied step by step between occurences of <*>.

<`$`>, which is fmap as an infix operator.
(<`$`>) :: (Functor f) => (a -> b) -> f a -> f b
f <`$`> x = fmap f x
By using <`$`>, the applicative style really shines, because now if we want to apply a function f between three applicative functors, we can write f <`$`> x <*> y <*> z. If the parameters weren't applicative functors but normal values, we'd write f x y z.

(++) <`$`> Just "johntra" <*> Just "volta"  
Just "johntravolta"
when we do (++) <`$`> Just "johntra" <*> Just "volta", first (++), which has a type of (++) :: [a] -> [a] -> [a] gets mapped over Just "johntra", resulting in a value that's the same as Just ("johntra"++) and has a type of Maybe ([Char] -> [Char]). Notice how the first parameter of (++) got eaten up and how the as turned into Chars. And now Just ("johntra"++) <*> Just "volta" happens, which takes the function out of the Just and maps it over Just "volta", resulting in Just "johntravolta". Had any of the two values been Nothing, the result would have also been Nothing.

In [80]:
(++) "johntra" "volta"

"johntravolta"

In [81]:
[(*0),(+100),(^2)] <*> [1,2,3]

[0,0,0,101,102,103,1,4,9]

In [82]:
[(+),(*)] <*> [1,2] <*> [3,4]

[4,5,5,6,3,4,6,8]

myAction :: IO String  
myAction = do  
    a <- getLine  
    b <- getLine  
    return `$` a ++ b
    
is same as

myAction :: IO String  
myAction = (++) <`$`> getLine <*> getLine

In [83]:
(\x y z -> [x,y,z]) <$> (+3) <*> (*2) <*> (/2) $ 5

[8.0,10.0,2.5]

Control.Applicative defines a function that's called liftA2, which has a type of liftA2 :: (Applicative f) => (a -> b -> c) -> f a -> f b -> f c

In [87]:
liftA2 :: (Applicative f) => (a -> b -> c) -> f a -> f b -> f c  
liftA2 f a b = f <$> a <*> b

liftA2 (:) (Just 3) (Just [4])  
Just [3,4]

## newtype
newtype is faster, just one constructor with one field, turning an existing type into a new type

## `type` vs `newtype` vs `data`
The type keyword is for making type synonyms.
The newtype keyword is for taking existing types and wrapping them in new types, mostly so that it's easier to make them instances of certain type classes. When we use newtype to wrap an existing type, the type that we get is separate from the original type; consider newtype declarations as data declarations that can only have one constructor and one field.
The data keyword is for making your own data types; They can have as many constructors and fields as you wish and can be used to implement any algebraic data type

## Monoids
A monoid is when you have an associative binary function and a value which acts as an identity with respect to that function. When something acts as an identity with respect to a function, it means that when called with that function and some other value, the result is always equal to that other value. 1 is the identity with respect to * and [] is the identity with respect to ++.

...
http://learnyouahaskell.com/a-fistful-of-monads#getting-our-feet-wet-with-maybe

https://www.haskell.org/tutorial/monads.html